머신 러닝의 근본적인 이슈는 최적화와 일반화 사이의 줄다리기입니다. **최적화 (optimization)**은 가능한 훈련 데이터에서 최고의 성능을 얻으려고 모델을 조정하는 과정입니다. 반면 **일반화 (generalization)**는 훈련된 모델이 이전에 본 적 없는 데이터에서 얼마나 잘 수행되는지 의미합니다. 모델을 만드는 목적은 좋은 일반화 성능을 얻는 것입니다. 하지만 일반화 성능을 제어할 방법은 없고 단진 훈련 데이터를 기반으로 모델을 조정하는 수만 있습니다.

훈련 데이터의 손실이 낮아질수록 테스트 데이터의 손실도 낮아집니다. 이런 상황이 발생했을때 모델이 **과소적합 (underfitting)**되었다고 말합니다. 모델의 성능이 계속 발전될 여지가 있습니다. 네트워크가 훈련 데이터에 있는 관련 특성을 모두 학습하지 못한 상황입니다. 하지만 훈련 데이터에 여러번 반복 학습하고 나면 어느 시점부터 일반화 성능이 더 이상 높아지지 않습니다. 검증 세트의 성능이 멈추고 감소되기 시작합니다. 즉 모델이 **과대적합 (overfitting)**되기 시작합니다. 이는 훈련 데이터에 특화된 패턴을 학습하기 시작했다는 의미입니다.

모델이 관련성이 없고 좋지 못한 패턴을 훈련 데이터에서 학습하지 못하도록 하려면 가장 좋은 방법은 더 많은 훈련 데이터를 모으는 것입니다. 더 많은 데이터에서 훈련된 모델은 자연히 일반화 성능이 더울 뛰어납니다. 더 많은 데이터를 모으는 것이 불가능할 때는 모델이 수용할 수 있는 정보의 양을 조절하거나 저장할 수 있는 정보에 제약을 가하는 것 입니다. 네트워크가 적은 수의 패턴만 기억할 수 있다면 최적화 과정에서 가장 중요한 패턴에 집중하게 될 것입니다.

이런 식으로 과대적합을 피하는 처리 과정을 **규제 (regularization)**라고 합니다.

### 네트워크 크기 축소

과대적합을 막는 가장 단순한 방법은 *모델의 크기*, 즉 모델에 있는 학습 파라미터의 수를 줄이는 것입니다. 파라미터의 수는 *층의 수*와 *각 층의 유닛 수*에 의해 결정됩니다. 당연하게 파라미터가 많은 모델이 기억 용량이 더 많습니다. 훈련 샘플과 타깃 사이를 딕셔너리 같은 일대일 매핑으로 완벽하게 학습할 수도 있습니다. 하지만 이런 매핑은 일반화 능력이 없습니다.

다른 한편으로 네트워크가 기억 용량에 제한이 있다면 이런 매핑을 쉽게 학습하지 못합니다. 따라서 손실을 최소화하기 위해 타깃에 대한 예층 성능을 가진 압축된 표현을 학습해야 합니다. 동시에 과소적합되지 않도록 충분한 파라미터를 가진 모델을 사용해야 합니다.

적절한 모델 크기를 찾는 이발적인 작업 흐름은 비교적 적은 수의 층과 파라미터로 시작합니다. 그 다음 검증 손실이 감소되기 시작할 때 까지 층이나 유닛의 수를 늘리는 것입니다.

##### IMDB 원본 모델

<br></br>
원래 네트워크는 다음과 같습니다.

```python
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation = 'relu', input_shape = (10000,)))
model.add(layers.Dense(16, activation = 'relu'))
model.add(layers.Dense(1, activation = 'sigmoid'))
```
<br></br>

##### 작은 용량의 모델

더 작은 네트워크로 바꾸어 봅시다.

```python
model = models.Sequential()
model.add(layers.Dense(6, activation = 'relu', input_shape = (10000,)))
model.add(layers.Dense(6, activation = 'relu'))
model.add(layers.Dense(1, activation = 'sigmoid'))
```

아래 그림은 원본 네트워크와 축소된 네트워크의 검증 손실을 비교한 것 입니다.
<br></br>
![](../images/4-4-model1.png)

<br></br>
작은 네트워크가 기본 네트워크보다 더 늦게 과대적합되기 시작했습니다.(4 vs 6). 과대적합이 시작되었을 때 성능이 더 천천히 감소되었습니다.

##### 큰 용량의 모델

```python
model = models.Sequential()
model.add(layers.Dense(1024, activation = 'relu', input_shape = (10000,)))
model.add(layers.Dense(1024, activation = 'relu'))
model.add(layers.Dense(1, activation = 'sigmoid'))
```
<br></br>
![](../images/4-4-model2.png)

<br></br>

더 큰 네트워크는 첫 번째 에포크 이후 거의 바로 과대적합이 시작되었고 갈수록 더 심해집니다. 검증 손실도 매우 불안합니다.

<br></br>
![](../images/4-4-model3.png)
<br></br>

두 네트워크의 훈련 손실을 보여줍니다. 용량이 큰 네트워크는 훈련 손실이 매우 빠르게 0 에 가까워집니다. 용량이 많은 네트워크일수록 더 빠르게 훈련 데이터를 모델링할 수 있지만 과대적합에 더 민감하게 반응합니다.

### 가중치 규제

**Occam's Razor**는 두 가지의 설명이 있다면 더 적은 가정이 필요한 간단한 설명이 옳을 것이라는 이론입니다. 신경망으로 학습되는 모델에도 적용됩니다. 어떤 네트워크 구조가 주어졌을대 데이터를 설명할 수 있는 가중치 값의 집합은 여러개 입니다. 간단한 모델이 복잡한 모델보다 덜 과대적합될 가능성이 높습니다.

여기에서 간단한 모델은 파라미터 값 분포의 엔트로피가 작은 모델입니다. 그러므로 과대적합을 완화하기 위한 일반적인 방법은 네트워크의 복잡도에 제한을 두어 가중치가 작은 값을 가지도록 강제하는 것입니다. 가중치 값의 분포가 더 균일하게 됩니다. 이를 **가중치 규제 (weight regularization)**라고 하며, 네트워크의 손실 함수에 큰 가중치에 연관된 비용을 추가합니다. 

* L1 Regularization: 가중치의 절대값에 비례하는 비용이 추가됩니다. (가중치의 *L1 Norm*)
* L2 Regularization: 가중치의 제곱에 비례하는 비용이 추가됩니다. (가중치의 *L2 Norm*)

##### 모델에 L2 가중치 추가하기

```python
from keras import regularizers

model = models.Sequential()
model.add(layers.Dense(16, kernel_regularizer = regularizers.l2(0.001),
                       activation = 'relu', input_shape = (10000,)))
model.add(layers.Dense(16, kernel_regularizer = regularizers.l2(0.001),
                       activation = 'relu'))
model.add(layers.Dense(1, activation = 'sigmoid'))
```

<br></br>

* `l2(0.001)`는 가중치 행렬의 모든 원소를 제곱하고 0.001를 곱하여 네트워크의 전체 손실에 더해진다는 의미입니다. 이 페널티는 훈련할때만 추가됩니다. 아래 그래프에서 볼 수 있듯 L2규제를 사용한 모델이 과대적합에 잘 견디고 있습니다.

<br></br>

![](../images/4-4-L2.png)


### 드랍아웃

**드랍아웃**은 신경망을 규제하는 방법중 가장 널리 사용되는 효과적인 방법입니다. 네트워크 층에 드롭아웃을 적용하면 훈련하는 동안 무작위로 층의 일부 출력 특성을 제외시킵니다. 

* [0.2, 0.5, 1.3, 0.8, 1.1] -> [0, 0.5, 1.3, 0, 1.1]
* 드랍아웃 비율은 0이 될 특성의 비율입니다. 
* 보통 0.2에서 0.5 사이로 지정됩니다.
* 테스트 단계에서는 드롭아웃되지 않습니다. 
* 대신 층의 출력을 드롭아웃 비율에 비례하여 줄여줍니다.

![](../images/4-5-dropout.png)

<br></br>

무계획적으로 보일 수 있습니다. 드롭아웃이 왜 과대적합을 줄이는데 도움이 될까요?
* 은행에서 부정행위를 방지하는 것 처럼, 뉴런의 일부를 무작위하게 제거하면 뉴런의 부정한 협업을 방지하고 과대적합을 감소시킴
* 층의 출력 값에 노이즈를 추가하여 중요하지 않은 우연한 패턴을 깨트리는 것
* 노이즈가 없다면 네트워크가 이 패턴을 기억하기 시작할 것

##### IMDB에 드롭아웃 추가하기

```python
model = models.Sequential()
model.add(layers.Dense(16, activation = 'relu', input_shape = (10000,)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(16, activation = 'relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation = 'sigmoid'))
```

<br></br>

* `l2(0.001)`는 가중치 행렬의 모든 원소를 제곱하고 0.001를 곱하여 네트워크의 전체 손실에 더해진다는 의미입니다. 이 페널티는 훈련할때만 추가됩니다. 아래 그래프에서 볼 수 있듯 L2규제를 사용한 모델이 과대적합에 잘 견디고 있습니다.

<br></br>

![](../images/4-4-dropout2.png)
